In [1]:
%load_ext dotenv
%dotenv -o

import os
import random
import multiprocessing
from datetime import datetime
from urllib.parse import urlsplit

import ujson
import psycopg2
import pandas as pd
from tqdm import tqdm
from peewee import Model
from newspaper import Article
from newspaper.article import ArticleDownloadState
from playhouse.shortcuts import model_to_dict
from playhouse.postgres_ext import PostgresqlExtDatabase, DateTimeField, CharField, TextField, IntegerField, fn

In [ ]:
## this stuff should be moved somewhere :) 

peewee_database = PostgresqlExtDatabase(os.environ['RESEARCHABLY_DB_NAME'], **{
    'host': os.environ    ['RESEARCHABLY_DB_HOST'],
    'user': os.environ    ['RESEARCHABLY_DB_USER'],
    'password': os.environ['RESEARCHABLY_DB_PASSWORD'],
    'register_hstore':    False
})


class BaseModel(Model):
    class Meta:
        database = peewee_database

        
class ScrapedPage(BaseModel):
    batch = IntegerField(null=False)
    url = CharField(null=False)
    html = TextField(null=False)
    inserted_at = DateTimeField(null=False, default=datetime.now())
    updated_at = DateTimeField(null=False, default=datetime.now())

    class Meta:
        db_table = 'fnr_scraped_pages'
        
        
def big_select_query(query, batch_size=2000):
    cursor = peewee_database._local.conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
    cursor_name = 'task_big_query' + str(random.randint(1, 1000 * 1000))
    cursor.execute('begin; declare ' + cursor_name + ' cursor for ' + query.sql()[0], query.sql()[1])

    while True:
        cursor.execute('fetch ' + str(batch_size) + ' from ' + cursor_name + ' ;')
        batch_results = cursor.fetchall()

        if len(batch_results) <= 0:
            break

        for row in batch_results:
            yield row

    cursor.close()

# Get scraped articles from database

In [ ]:
scraped_pages_count = list(ScrapedPage.select(fn.Count(ScrapedPage.id).alias('count')).where(ScrapedPage.batch == 2))
scraped_pages_count = scraped_pages_count[0].count
scraped_pages_count

In [ ]:
peewee_database.connect()

In [ ]:
with open('data/7_opensources_co/scraped_pages.jsonl', 'w') as _out:
    for page in tqdm(big_select_query(ScrapedPage.select().where(ScrapedPage.batch == 2), 100)):
        _out.write(ujson.dumps(page) + '\n')

In [ ]:
1

# Clean articles

In [4]:
def parse_article(line):
    try:
        page = ujson.loads(line)
        article = Article(page['url'])
        article.set_html(page['html'])
        article.config.fetch_images = False
        article.parse()

        return ujson.dumps({
            'id': page['id'],
            'batch': page['batch'],
            'url': page['url'],
            'title': article.title,
            'content': article.text,
            'published_at': article.publish_date,
            'authors': article.authors,
            'meta_description': article.meta_description,
            'meta_keywords': article.meta_keywords
        })
    except Exception as a:
        print(a)
        return None

In [5]:
def yield_scraped_pages():
    with open('data/7_opensources_co/scraped_pages.jsonl', 'r') as _in:
        for line in _in:
            yield line

with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
    with open('data/7_opensources_co/scraped_pages_articles.jsonl', 'w') as _out:
        for result in tqdm(pool.imap(parse_article, yield_scraped_pages(), chunksize=100)):
            if result is None:
                continue

            _out.write(result + '\n')


0it [00:00, ?it/s]
1it [00:11, 11.83s/it]
101it [00:13,  7.50it/s]
101it [00:26,  3.86it/s]
201it [01:05,  3.05it/s]
5501it [01:06, 82.26it/s]
5672it [01:08, 83.29it/s]
5796it [01:11, 81.53it/s]
5884it [01:11, 81.97it/s]
5950it [01:12, 82.57it/s]
6003it [01:12, 82.45it/s]
6301it [01:13, 85.70it/s]
6401it [01:14, 86.14it/s]
6701it [01:14, 89.43it/s]
6801it [01:17, 88.07it/s]
6901it [01:17, 88.91it/s]
7001it [01:19, 87.71it/s]
7201it [01:21, 88.32it/s]
7301it [01:24, 86.78it/s]
7501it [01:24, 88.44it/s]
7701it [01:27, 88.11it/s]
8001it [01:28, 90.69it/s]
8301it [01:29, 92.48it/s]
8401it [01:31, 91.73it/s]
8501it [01:32, 92.14it/s]
8601it [01:33, 92.12it/s]
8801it [01:35, 92.25it/s]
8901it [01:37, 90.85it/s]
9001it [01:38, 91.13it/s]
9201it [01:39, 92.67it/s]
9301it [01:41, 91.22it/s]
9401it [01:42, 91.90it/s]
9601it [01:42, 93.71it/s]
9901it [01:44, 94.92it/s]
10001it [01:46, 94.30it/s]
10201it [01:47, 94.78it/s]
10301it [01:48, 95.12it/s]
10401it [01:49, 95.28it/s]
10501it [01:52, 93.4

44601it [06:48, 109.31it/s]
44801it [06:49, 109.34it/s]
44901it [06:50, 109.50it/s]
45001it [06:50, 109.56it/s]
45101it [06:50, 109.73it/s]
45201it [06:52, 109.55it/s]
45301it [06:52, 109.73it/s]
45401it [06:53, 109.70it/s]
45601it [06:54, 110.04it/s]
45701it [06:54, 110.23it/s]
45801it [06:56, 110.06it/s]
45901it [06:56, 110.15it/s]
46001it [06:58, 109.94it/s]
46101it [06:59, 109.91it/s]
46201it [07:01, 109.53it/s]
46401it [07:03, 109.52it/s]
46501it [07:03, 109.72it/s]
46601it [07:04, 109.79it/s]
46801it [07:06, 109.66it/s]
47001it [07:07, 110.00it/s]
47201it [07:07, 110.42it/s]
47301it [07:07, 110.57it/s]
47401it [07:08, 110.52it/s]
47501it [07:09, 110.49it/s]
47601it [07:11, 110.26it/s]
47701it [07:13, 110.15it/s]
47801it [07:14, 109.89it/s]
47901it [07:15, 110.08it/s]
48001it [07:16, 109.87it/s]
48301it [07:17, 110.33it/s]
48401it [07:19, 110.07it/s]
48501it [07:20, 110.21it/s]
48701it [07:20, 110.51it/s]
48901it [07:21, 110.87it/s]
49001it [07:22, 110.83it/s]
49101it [07:22, 110.

81801it [12:12, 111.66it/s]
81901it [12:13, 111.65it/s]
82001it [12:14, 111.65it/s]
82101it [12:16, 111.46it/s]
82201it [12:18, 111.38it/s]
82301it [12:18, 111.37it/s]
82401it [12:19, 111.48it/s]
82501it [12:19, 111.51it/s]
82601it [12:19, 111.62it/s]
82701it [12:21, 111.57it/s]
82801it [12:21, 111.66it/s]
82901it [12:22, 111.59it/s]
83301it [12:23, 112.01it/s]
83401it [12:25, 111.81it/s]
83501it [12:27, 111.76it/s]
83601it [12:27, 111.83it/s]
83701it [12:29, 111.72it/s]
83801it [12:29, 111.75it/s]
83901it [12:31, 111.69it/s]
84001it [12:31, 111.78it/s]
84101it [12:32, 111.82it/s]
84201it [12:32, 111.87it/s]
84301it [12:34, 111.80it/s]
84401it [12:34, 111.84it/s]
84501it [12:36, 111.67it/s]
84701it [12:37, 111.85it/s]
84801it [12:37, 111.88it/s]
84901it [12:39, 111.84it/s]
85001it [12:40, 111.79it/s]
85101it [12:41, 111.73it/s]
85201it [12:41, 111.82it/s]
85301it [12:43, 111.67it/s]
85401it [12:44, 111.73it/s]
85501it [12:45, 111.67it/s]
85601it [12:45, 111.78it/s]
85701it [12:46, 111.

129101it [17:41, 121.66it/s]
129501it [17:41, 122.00it/s]
129801it [17:41, 122.27it/s]
130001it [17:41, 122.43it/s]
130110it [17:41, 122.52it/s]
130213it [17:42, 122.54it/s]
130401it [17:43, 122.60it/s]
130601it [17:47, 122.30it/s]
130901it [17:48, 122.54it/s]
131201it [17:48, 122.78it/s]
131601it [17:48, 123.14it/s]
131719it [17:49, 123.21it/s]
131813it [17:49, 123.26it/s]
131901it [17:49, 123.29it/s]
132001it [17:50, 123.31it/s]
132101it [17:51, 123.34it/s]
132201it [17:54, 122.98it/s]
132501it [17:55, 123.24it/s]
132601it [17:55, 123.32it/s]
132801it [17:55, 123.43it/s]
133301it [17:56, 123.82it/s]
133501it [17:57, 123.94it/s]
133601it [17:57, 124.01it/s]
133701it [17:57, 124.03it/s]
133801it [18:01, 123.67it/s]
134001it [18:02, 123.83it/s]
134101it [18:02, 123.91it/s]
134601it [18:02, 124.31it/s]
134701it [18:03, 124.37it/s]
134901it [18:03, 124.52it/s]
135001it [18:03, 124.57it/s]
135101it [18:04, 124.55it/s]
135301it [18:05, 124.68it/s]
135401it [18:09, 124.31it/s]
135901it [18:0

176601it [23:09, 127.10it/s]
176701it [23:13, 126.81it/s]
176801it [23:14, 126.77it/s]
177201it [23:16, 126.91it/s]
177801it [23:18, 127.16it/s]
178101it [23:22, 127.00it/s]
178201it [23:23, 126.94it/s]
178301it [23:28, 126.63it/s]
178501it [23:30, 126.52it/s]
179301it [23:31, 126.99it/s]
179401it [23:32, 127.04it/s]
179701it [23:35, 126.98it/s]
179801it [23:38, 126.73it/s]
180001it [23:39, 126.76it/s]
180201it [23:41, 126.80it/s]
180301it [23:41, 126.86it/s]
180401it [23:43, 126.73it/s]
180701it [23:44, 126.85it/s]
181301it [23:46, 127.14it/s]
181401it [23:47, 127.05it/s]
181501it [23:49, 126.95it/s]
181801it [23:51, 127.03it/s]
181901it [23:51, 127.07it/s]
182101it [23:54, 126.96it/s]
182201it [23:55, 126.90it/s]
182301it [23:56, 126.94it/s]
182601it [23:57, 127.01it/s]
182901it [23:59, 127.08it/s]
183001it [24:00, 127.00it/s]
183201it [24:03, 126.88it/s]
183301it [24:04, 126.91it/s]
183601it [24:04, 127.08it/s]
183701it [24:07, 126.88it/s]
183801it [24:08, 126.91it/s]
184001it [24:0

...

1212601it [2:37:28, 128.34it/s]
1212801it [2:37:29, 128.34it/s]
1212901it [2:37:30, 128.34it/s]
1213201it [2:37:31, 128.36it/s]
1213301it [2:37:33, 128.35it/s]
1213401it [2:37:35, 128.33it/s]
1213501it [2:37:35, 128.34it/s]
1213701it [2:37:35, 128.35it/s]
1213801it [2:37:37, 128.34it/s]
1214101it [2:37:38, 128.37it/s]
1214201it [2:37:39, 128.35it/s]
1214301it [2:37:40, 128.35it/s]
1214401it [2:37:42, 128.34it/s]
1214701it [2:37:43, 128.35it/s]
1214801it [2:37:43, 128.36it/s]
1214901it [2:37:46, 128.34it/s]
1215101it [2:37:46, 128.36it/s]
1215201it [2:37:48, 128.34it/s]
1215301it [2:37:49, 128.34it/s]
1215401it [2:37:49, 128.35it/s]
1215601it [2:37:51, 128.34it/s]
1215801it [2:37:52, 128.36it/s]
1215901it [2:37:53, 128.35it/s]
1216101it [2:37:54, 128.36it/s]
1216301it [2:37:56, 128.34it/s]
1216501it [2:38:00, 128.32it/s]
1216801it [2:38:05, 128.28it/s]
1217701it [2:38:06, 128.36it/s]
1218001it [2:38:08, 128.36it/s]
1218101it [2:38:11, 128.33it/s]
1218301it [2:38:13, 128.34it/s]
1218401i

You must `download()` an article first!




1227601it [2:39:32, 128.24it/s]
1227701it [2:39:33, 128.24it/s]
1227901it [2:39:33, 128.26it/s]
1228001it [2:39:37, 128.22it/s]
1228101it [2:39:40, 128.18it/s]
1228201it [2:39:41, 128.19it/s]
1228401it [2:39:42, 128.19it/s]
1228601it [2:39:43, 128.20it/s]
1228701it [2:39:44, 128.20it/s]
1229101it [2:39:45, 128.23it/s]
1229201it [2:39:45, 128.24it/s]
1229301it [2:39:47, 128.22it/s]
1229601it [2:39:49, 128.23it/s]
1229701it [2:39:54, 128.17it/s]
1229801it [2:39:54, 128.18it/s]
1229901it [2:39:55, 128.18it/s]
1230001it [2:39:56, 128.17it/s]
1230101it [2:39:56, 128.18it/s]
1230301it [2:39:57, 128.19it/s]
1230501it [2:39:57, 128.20it/s]
1230601it [2:39:58, 128.20it/s]
1230701it [2:39:59, 128.21it/s]
1230801it [2:40:00, 128.21it/s]
1230901it [2:40:00, 128.22it/s]
1231001it [2:40:00, 128.22it/s]
1231101it [2:40:01, 128.21it/s]
1231201it [2:40:03, 128.20it/s]
1231301it [2:40:09, 128.14it/s]
1231501it [2:40:09, 128.15it/s]
1231601it [2:40:10, 128.16it/s]
1231701it [2:40:10, 128.16it/s]
1231801

You must `download()` an article first!

You must `download()` an article first!




1256334it [2:44:21, 127.40it/s]
1257501it [2:44:23, 127.49it/s]
1257601it [2:44:24, 127.49it/s]
1258301it [2:44:26, 127.53it/s]
1258401it [2:44:26, 127.54it/s]
1258501it [2:44:28, 127.52it/s]
1258601it [2:44:30, 127.51it/s]
1258701it [2:44:32, 127.50it/s]
1259001it [2:44:32, 127.52it/s]
1259101it [2:44:34, 127.51it/s]
1259401it [2:44:37, 127.51it/s]
1259701it [2:44:37, 127.54it/s]
1259901it [2:44:41, 127.51it/s]
1260401it [2:44:45, 127.50it/s]
1261301it [2:44:45, 127.59it/s]
1261401it [2:44:47, 127.58it/s]
1261501it [2:44:47, 127.59it/s]
1261601it [2:44:48, 127.59it/s]
1261701it [2:44:49, 127.58it/s]
1261801it [2:44:49, 127.58it/s]
1261901it [2:44:51, 127.58it/s]
1262001it [2:44:53, 127.56it/s]
1262201it [2:44:56, 127.55it/s]
1263101it [2:44:56, 127.63it/s]
1263201it [2:44:57, 127.62it/s]
1263301it [2:44:58, 127.62it/s]
1263401it [2:44:59, 127.62it/s]
1263501it [2:45:00, 127.61it/s]
1263701it [2:45:02, 127.62it/s]
1263801it [2:45:03, 127.61it/s]
1263901it [2:45:04, 127.60it/s]
1264401

# Analyse the articles

In [6]:
pages = []
with open('data/7_opensources_co/scraped_pages_articles.jsonl', 'r') as _in:
    for line in tqdm(_in):
        pages.append(ujson.loads(line))

df_pages = pd.DataFrame(pages)
df_pages


0it [00:00, ?it/s]
4851it [00:00, 48124.87it/s]
11394it [00:00, 56774.05it/s]
17626it [00:00, 58468.59it/s]
21583it [00:00, 52868.03it/s]
27683it [00:00, 54481.40it/s]
35426it [00:00, 58190.16it/s]
44486it [00:00, 62802.12it/s]
51330it [00:00, 57282.27it/s]
57022it [00:01, 56461.94it/s]
64542it [00:01, 58152.91it/s]
71642it [00:01, 59192.83it/s]
78564it [00:01, 59975.76it/s]
85116it [00:01, 55843.22it/s]
91198it [00:01, 56148.80it/s]
96876it [00:01, 54998.25it/s]
102091it [00:01, 54708.23it/s]
107616it [00:01, 54743.28it/s]
113417it [00:02, 51236.06it/s]
120942it [00:02, 52265.69it/s]
133576it [00:02, 55333.87it/s]
148769it [00:02, 59173.11it/s]
158587it [00:02, 55380.28it/s]
166302it [00:03, 54544.86it/s]
172951it [00:03, 53687.88it/s]
178762it [00:03, 53010.94it/s]
183988it [00:03, 52486.52it/s]
188821it [00:03, 52052.24it/s]
193392it [00:03, 51544.36it/s]
197694it [00:03, 50996.08it/s]
201742it [00:04, 47480.81it/s]
206280it [00:04, 47430.38it/s]
211143it [00:04, 47456.02it/s]
2169

authors  batch  \
0                                                       []      2   
1                                                       []      2   
2                                                       []      2   
3                                         [Vanessa Beeley]      2   
4                                         [Kate Anslinger]      2   
5                                          [Brandi Philip]      2   
6                                                       []      2   
7                                                       []      2   
8                             [John Aravosis, Chris Andoe]      2   
9        [Alexander Smith, Robert Winthrop, Kate Anslin...      2   
10       [Ruth Harris, Wendy Michaels, Alexander Smith,...      2   
11       [Wendy Michaels, Ruth Harris, Kate Anslinger, ...      2   
12       [Alexander Smith, Ruth Harris, Kate Anslinger,...      2   
13       [Kate Anslinger, Alexander Smith, Ruth Harris,...      2   
14       [Alexander Smith, Kate Anslinger, Ruth Harris,...      2   
15                                        [Kate Anslinger]      2   
16                                                      []      2   
17                                                      []      2   
18                                                      []      2   
19                                                      []      2   
20                                                      []      2   
21                                                      []      2   
22                                                      []      2   
23                                                      []      2   
24                                             [Pol Pinoy]      2   
25                                                      []      2   
26                                                      []      2   
27                                                      []      2   
28                                               [Shel By]      2   
29                                                      []      2   
...                                                    ...    ...   
1265063                                                 []      2   
1265064                                                 []      2   
1265065                                                 []      2   
1265066                                                 []      2   
1265067                                                 []      2   
1265068                                                 []      2   
1265069                                                 []      2   
1265070                                                 []      2   
1265071                                                 []      2   
1265072                                                 []      2   
1265073                                                 []      2   
1265074                                                 []      2   
1265075                                                 []      2   
1265076                                                 []      2   
1265077                                                 []      2   
1265078                                                 []      2   
1265079                                                 []      2   
1265080                                                 []      2   
1265081                                                 []      2   
1265082                                                 []      2   
1265083                                                 []      2   
1265084                                                 []      2   
1265085                                                 []      2   
1265086                                                 []      2   
1265087                                                 []      2   
1265088                                                 []      2   
1265089                                          

In [13]:
netlocs_count = {}
with tqdm(total=len(df_pages)) as progress:
    for url in df_pages.url:
        netloc = urlsplit(url).netloc
        if netloc not in netlocs_count:
            netlocs_count[netloc] = 0

        netlocs_count[netloc] += 1
        progress.update()


100%|██████████| 1265093/1265093 [00:07<00:00, 159191.80it/s]

In [18]:
df_netlocs_count = pd.DataFrame([{'domain': k, 'count': v} for k, v in netlocs_count.items()]).sort_values('count', ascending=False)
df_netlocs_count

count                      domain
115  165817            www.dailykos.com
463   99444           www.express.co.uk
423   90977             sputniknews.com
131   71362           www.lifezette.com
332   62211           beforeitsnews.com
75    26395       www.judicialwatch.org
107   26136           www.breitbart.com
257   20840             dailycaller.com
248   20819      www.abovetopsecret.com
226   18339  www.washingtonexaminer.com
447   17719         us.blastingnews.com
267   17438  www.chroniclesmagazine.org
486   17421                 katehon.com
169   17062           www.zerohedge.com
151   16779                 twitchy.com
42    15901     investmentwatchblog.com
101   15037         www.lewrockwell.com
315   14226            www.rawstory.com
242   13961               www.pravda.ru
69    12938         www.naturalnews.com
34    12403            www.redstate.com
370   12057           www.wikileaks.org
119   11893            www.theonion.com
122   11132      www.weeklystandard.com
143   10456       www.conservapedia.com
304    9934              wikispooks.com
381    9689             shadowproof.com
276    9377             www.newsmax.com
484    9188     www.teapartytribune.com
170    9070              guardianlv.com
..      ...                         ...
418       1           www.anonjekloy.tk
424       1              libertytalk.fm
322       1          AutomaticEarth.com
465       1        patriotnewsdaily.com
300       1             usatoday.com.co
468       1                  bients.com
435       1       www.thecivilian.co.nz
162       1             newswire-24.com
163       1          www.zengardner.com
172       1      www.silverbearcafe.com
491       1                   ruptly.tv
175       1              amusmentic.com
489       1     www.betootaadvocate.com
180       1           patriotupdate.com
186       1         www.callthecops.net
189       1         bigamericannews.com
192       1         TheRacketReport.com
209       1           dcclothesline.com
211       1            beehivebugle.com
216       1              breakpoint.org
218       1      blackinusrancenews.com
478       1                adflegal.org
241       1              Alt-Market.com
243       1           dailybuzzlive.com
247       1       www.channel18news.com
273       1                www.gaia.com
471       1        www.BrotherJohnF.com
278       1           rightsidenews.com
279       1     countdowntozerotime.com
0         1       www.gatesofvienna.net

[555 rows x 2 columns]

In [39]:
df_pages[df_pages.url.str.contains('www.express.co.uk')]

authors  batch  \
1550         [Charlie Bayliss Exclusive, Karin Von Hippel]      2   
1551                         [Alix Culbertson, Lord Hague]      2   
1562                                                    []      2   
1563                                                    []      2   
1571                                                    []      2   
1610                         [Nicole Stinson, Theresa May]      2   
1611                               [Taryn Tarrant-Cornish]      2   
1612                                          [Matt Drake]      2   
1615                  [Rebecca Perring, Lady Pamela Hicks]      2   
1616                                      [Prince Charles]      2   
1617                           [Nicole Stinson, Liu Cixin]      2   
1618                                          [Matt Drake]      2   
1619                                        [Joseph Carey]      2   
1620                                          [Matt Drake]      2   
1621                                         [Sean Martin]      2   
1624                                         [Thomas Hunt]      2   
1635                                                    []      2   
1636                             [Darren Hunt, Lord Sugar]      2   
1637                      [Joe Barnes, Christopher Harvey]      2   
1638                          [Simon Osborne, Mark Failla]      2   
1640                                      [Becca Longmire]      2   
1643                                   [Sebastian Kettley]      2   
1644                         [Jon Austin, Daniel, D Alien]      2   
1649                             [Jon Austin, David Meade]      2   
1659                                      [Becca Longmire]      2   
1660                                      [Becca Longmire]      2   
1661     [Becca Longmire, Sharon Stone Talks Maintainin...      2   
1662          [Roxanne Hughes, Bradley Walsh On The Chase]      2   
1688                         [Julia Kuttner, Debbie Mcgee]      2   
1689                                     [Camilla Tominey]      2   
...                                                    ...    ...   
1262477                                       [Gary Jones]      2   
1262478                                      [Liam Martin]      2   
1262479                                      [Liam Martin]      2   
1262480                                       [Gary Jones]      2   
1262481                                      [Liam Martin]      2   
1262483                                      [Liam Martin]      2   
1262484                                       [Gary Jones]      2   
1262485                                 [Dion Dassanayake]      2   
1262486                                      [Liam Martin]      2   
1262487                                      [Liam Martin]      2   
1262488                                 [Dion Dassanayake]      2   
1262489                                       [Gary Jones]      2   
1262490                                      [Liam Martin]      2   
1262491                                       [Gary Jones]      2   
1262493                                       [Gary Jones]      2   
1262495                                      [Ben Pringle]      2   
1262499                                       [Gary Jones]      2   
1262501                                  [James Cambridge]      2   
1262502                                       [Gary Jones]      2   
1262506                                 [Dion Dassanayake]      2   
1262508                                 [Dion Dassanayake]      2   
1262512                                       [Gary Jones]      2   
1263485                                        [Oli Smith]      2   
1263487  [Jo Wiley, Researchers Barts, London School Of...      2   
1263490                  [John Ingham, Dr Barbora Ondrova]      2   
1263492    [Rebecca Perring, Gerard Couzens, Naghmeh King]      2   
1263493                     [Dion Dassanayake, Br

# Extract the urls for continuing of scraping

In [40]:
with open('data/7_opensources_co/scraped_pages_urls.json', 'w') as _out: 
    ujson.dump(list(df_pages.url.values), _out)